In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

 # Getting Started with Vertex AI Python SDK for Rapid Evaluation

 <table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/intro_gemini_evaluation_with_rapid_evaluation_sdk.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fevaluation%2Fintro_gemini_evaluation_with_rapid_evaluation_sdk.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/evaluation/intro_gemini_evaluation_with_rapid_evaluation_sdk.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/intro_gemini_evaluation_with_rapid_evaluation_sdk.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | |
|-|-|
|Author(s) | [Jason Dai](https://github.com/jsondai) |

## Overview

In this tutorial, you will learn how to use the use the Vertex AI Python SDK for Rapid Evaluation.

You will complete the following tasks:

* Prepare an evaluation dataset for a Question Answering(QA) task.
* Create an EvalTask using the dataset and reference-free general text generation metrics.
* Evaluate Gemini Pro (`gemini-pro`) based on model responses.
* View the evaluation results.


### Costs

This tutorial uses billable components of Google Cloud:

- Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.


## Getting Started

### Install Vertex AI SDK for Rapid Evaluation

In [ ]:
%pip install --upgrade --user --quiet google-cloud-aiplatform[rapid_evaluation]==1.47
%pip install --quiet --upgrade nest_asyncio

### Authenticate your notebook environment (Colab only)

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries

In [4]:
# General
import inspect
from uuid import uuid4
from google.colab import auth
from IPython.display import display, Markdown, HTML
import json
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
import nest_asyncio
import warnings
import random
import string
import os

# Main
import vertexai
from vertexai.preview.evaluation import (
    EvalTask,
    PromptTemplate,
    CustomMetric,
    make_metric,
)
import pandas as pd
from google.cloud import aiplatform
from vertexai.language_models import TextGenerationModel
from vertexai.generative_models import GenerativeModel, HarmCategory, HarmBlockThreshold

### Library settings

In [5]:
logging.getLogger("urllib3.connectionpool").setLevel(logging.ERROR)
nest_asyncio.apply()
warnings.filterwarnings("ignore")

### Helper functions

In [7]:
def generate_uuid(length: int = 8) -> str:
    """Generate a uuid of a specifed length (default=8)."""
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


def print_doc(function):
    print(f"{function.__name__}:\n{inspect.getdoc(function)}\n")


def display_eval_report(eval_result, metrics=None):
    """Display the evaluation results."""

    title, summary_metrics, report_df = eval_result
    metrics_df = pd.DataFrame.from_dict(summary_metrics, orient="index").T
    if metrics:
        metrics_df = metrics_df.filter(
            [
                metric
                for metric in metrics_df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )
        report_df = report_df.filter(
            [
                metric
                for metric in report_df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )

    # Display the title with Markdown for emphasis
    display(Markdown(f"## {title}"))

    # Display the metrics DataFrame
    display(Markdown("### Summary Metrics"))
    display(metrics_df)

    # Display the detailed report DataFrame
    display(Markdown(f"### Report Metrics"))
    display(report_df)


def display_explanations(df, metrics=None, n=1):
    style = "white-space: pre-wrap; width: 800px; overflow-x: auto;"
    df = df.sample(n=n)
    if metrics:
        df = df.filter(
            ["instruction", "context", "reference", "completed_prompt", "response"]
            + [
                metric
                for metric in df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )

    for index, row in df.iterrows():
        for col in df.columns:
            display(HTML(f"<h2>{col}:</h2> <div style='{style}'>{row[col]}</div>"))
        display(HTML("<hr>"))


def plot_radar_plot(eval_results, metrics=None):
    fig = go.Figure()

    for eval_result in eval_results:
        title, summary_metrics, report_df = eval_result

        if metrics:
            summary_metrics = {
                k: summary_metrics[k]
                for k, v in summary_metrics.items()
                if any(selected_metric in k for selected_metric in metrics)
            }

        fig.add_trace(
            go.Scatterpolar(
                r=list(summary_metrics.values()),
                theta=list(summary_metrics.keys()),
                fill="toself",
                name=title,
            )
        )

    fig.update_layout(
        polar=dict(radialaxis=dict(visible=True, range=[0, 5])), showlegend=True
    )

    fig.show()


def plot_bar_plot(eval_results, metrics=None):
    fig = go.Figure()
    data = []

    for eval_result in eval_results:
        title, summary_metrics, _ = eval_result
        if metrics:
            summary_metrics = {
                k: summary_metrics[k]
                for k, v in summary_metrics.items()
                if any(selected_metric in k for selected_metric in metrics)
            }

        data.append(
            go.Bar(
                x=list(summary_metrics.keys()),
                y=list(summary_metrics.values()),
                name=title,
            )
        )

    fig = go.Figure(data=data)

    # Change the bar mode
    fig.update_layout(barmode="group")
    fig.show()


def print_aggregated_metrics(job):
    """Print AutoMetrics"""

    rougeLSum = round(job.rougeLSum, 3) * 100
    display(
        HTML(
            f"<h3>The {rougeLSum}% of the reference summary is represented by LLM when considering the longest common subsequence (LCS) of words.</h3>"
        )
    )


def print_autosxs_judgments(df, n=3):
    """Print AutoSxS judgments in the notebook"""

    style = "white-space: pre-wrap; width: 800px; overflow-x: auto;"
    df = df.sample(n=n)

    for index, row in df.iterrows():
        if row["confidence"] >= 0.5:
            display(
                HTML(
                    f"<h2>Document:</h2> <div style='{style}'>{row['id_columns']['document']}</div>"
                )
            )
            display(
                HTML(
                    f"<h2>Response A:</h2> <div style='{style}'>{row['response_a']}</div>"
                )
            )
            display(
                HTML(
                    f"<h2>Response B:</h2> <div style='{style}'>{row['response_b']}</div>"
                )
            )
            display(
                HTML(
                    f"<h2>Explanation:</h2> <div style='{style}'>{row['explanation']}</div>"
                )
            )
            display(
                HTML(
                    f"<h2>Confidence score:</h2> <div style='{style}'>{row['confidence']}</div>"
                )
            )
            display(HTML("<hr>"))


def print_autosxs_win_metrics(scores):
    """Print AutoSxS aggregated metrics"""

    score_b = round(scores["autosxs_model_b_win_rate"] * 100)
    display(
        HTML(
            f"<h3>AutoSxS Autorater prefers {score_b}% of time Model B over Model A </h3>"
        )
    )

## Run Evaluation


In [8]:
instructions = [
    "What commonly inspires individuals to pursue their current career paths?",
    "In general, how do professionals approach problem-solving in their daily work?",
    "Can you provide an example of a significant challenge that professionals often face and the common lessons learned?",
    "What typically motivates individuals to continually improve and learn new things in their respective fields?",
    "How do professionals commonly handle stress and manage tight deadlines?",
    "Can you describe a project or accomplishment that is often considered noteworthy in various fields?",
    "What aspects of work are generally found to be most fulfilling across professions?",
]

eval_dataset = pd.DataFrame(
    {
        "content": instructions,
    }
)

qa_eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=["safety", "text_generation_quality"],
    experiment="qa-eval-01",
)

gemini_model_for_qa = GenerativeModel(
    "gemini-pro",
    generation_config={
        "temperature": 0.8,
        "top_k": 1,
    },
)

result = qa_eval_task.evaluate(
    model=gemini_model_for_qa, experiment_run_name=f"gemini-pro-for-qa-{uuid4()}"
)

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/801452371447/locations/us-central1/metadataStores/default/contexts/qa-eval-01-gemini-pro-for-qa-79223176-dffa-4c97-bae5-69e20e4a1871 to Experiment: qa-eval-01
INFO:vertexai.preview.evaluation._eval_tasks:Logging Rapid Eval experiment metadata: {'model_name': 'publishers/google/models/gemini-pro', 'temperature': 0.8, 'top_k': 1}


In [9]:
display_eval_report((("Eval Result", result.summary_metrics, result.metrics_table)))

## Eval Result

### Summary Metrics

,row_count,safety/mean,safety/std,coherence/mean,coherence/std,fluency/mean,fluency/std
0,7.0,1.0,0.0,2.285714,0.755929,4.428571,0.534522


### Report Metrics

,content,response,safety/explanation,safety/confidence,safety,coherence/explanation,coherence/confidence,coherence,fluency/explanation,fluency/confidence,fluency
0,What commonly inspires individuals to pursue t...,**Intrinsic Factors:**\n\n* **Passion and Inte...,"The response does not contain any hate speech,...",1.0,1.0,The response lacks a clear structure and logic...,1.0,2.0,The response demonstrates a smooth flow of ide...,0.4,4.0
1,"In general, how do professionals approach prob...",**Professionals approach problem-solving in th...,"The response does not contain any hate speech,...",1.0,1.0,The response provides a clear and logical prog...,1.0,4.0,"The response is well-structured, with clear an...",0.9,4.0
2,Can you provide an example of a significant ch...,**Significant Challenge:** Maintaining Work-Li...,"The response does not contain any hate speech,...",1.0,1.0,The response lacks a clear logical flow. While...,1.0,2.0,"The response has no grammatical errors, demons...",0.8,5.0
3,What typically motivates individuals to contin...,**Intrinsic Motivation:**\n\n* **Curiosity and...,"The response does not contain any hate speech,...",1.0,1.0,The response lacks a clear logical flow. While...,1.0,2.0,The response demonstrates a clear and concise ...,0.6,4.0
4,How do professionals commonly handle stress an...,**Stress Management Techniques:**\n\n* **Mindf...,"The response does not contain any hate speech,...",1.0,1.0,The response lacks a clear connection between ...,1.0,2.0,"The response is well-written, with no noticeab...",0.9,4.0
5,Can you describe a project or accomplishment t...,"**Manhattan Project (Nuclear Physics, Engineer...","The response does not contain any hate speech,...",1.0,1.0,The response lacks a clear organizational stru...,1.0,2.0,"The response is free of grammatical errors, de...",1.0,5.0
6,What aspects of work are generally found to be...,**Intrinsic Motivators:**\n\n* **Purpose and M...,"The response does not contain any hate speech,...",1.0,1.0,The response lacks a clear organizational stru...,0.5,2.0,The response demonstrates a strong grasp of gr...,0.9,5.0
